# Regression model for IBNR estimates

In [5]:
import os
import itertools
import pandas as pd
import re
import math
import time
import numpy as np

from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

In [127]:
def columnas(valores,variable):
    y = [re.findall("\\d+", j)[0] for j in valores]
    y = [int(i) for i in y]
    todas = list(set(y))
    df = pd.DataFrame()
    df[f"y_{variable}"] = y
    for k in todas:
        #print(k)
        df[f"{variable}_{k}"] = ([1 if k == j else 0 for j in y])
    return df

def matrix_X(df_triangulo):
    k = len(df_triangulo.columns)
    alpha = [f'a_{i}' for i in range(1,k+1)]
    mu    = [f'u_{i}' for i in range(1,k+1)]
    lists = [alpha, mu]
    df    = pd.DataFrame(list(itertools.product(*lists)), columns=['a', 'u'])

    alpha    = columnas(valores  = df.a, variable = 'a')
    mu       = columnas(valores=df.u, variable = 'u')
    df_col= pd.concat([alpha, mu], axis=1)


    df_col['y_a'] = df_col['y_a'].astype(str) + df_col['y_u'].astype(str) 
    df_col['y_a'] = [int(i) for i in df_col['y_a']]
    df_col = df_col.drop(['y_u', 'u_1'], axis=1)
    df_col['a_1'] = 1
    df_col.rename(columns={'a_1': 'b0'}, inplace=True)
    df_col.rename(columns={'y_a': 'y_ii'}, inplace=True)
    #df_col = df_col.drop(['y_ii'], axis=1)
    return df_col


def matrix_y(df_triangulo):
    k = len(df_triangulo.columns)
    d0 = pd.DataFrame()
    for i in range(k):
        for j in range(k):
            d1 = pd.DataFrame({'y_ii': [int(f'{i+1}{j+1}')], 'Y': [math.log(df_triangulo.iloc[i, j])]})
            d0 = pd.concat([d0, d1], axis=0)
    return d0

def triangulo(df, grcode, entreno):
    
    if entreno:
        df_trinagulo = df[(df['GRCODE']== grcode ) & (df['DevelopmentYear']<=1997)].copy()
    else: 
        df_trinagulo = df[df['GRCODE']== grcode].copy()
        
    df_g         = df_trinagulo.groupby(["AccidentYear", "DevelopmentLag"]).agg({'IncurLoss_B': ['max']})
    df_g.columns = ['Pagos']
    df_g         = df_g.reset_index()
    pivot_data   = df_g.pivot(index='AccidentYear',columns='DevelopmentLag',values='Pagos').reset_index()
    pivot_data   = pivot_data.drop('AccidentYear', axis=1).cumsum(axis=1)
    
    return pivot_data


In [128]:
input = pd.read_csv(os.path.normpath(os.getcwd() + os.sep + os.pardir)+"/data/ppauto_pos.csv")

input = input[input.DevelopmentYear <= 1997]

cleaning_cond = np.array(['Adriatic Ins Co', 'Aegis Grp', 'Agency Ins Co Of MD Inc',
       'Allegheny Cas Co', 'American Modern Ins Grp Inc',
       'Armed Forces Ins Exchange', 'Auto Club South Ins Co',
       'Baltica-Skandinavia Rein Co Of Amer', 'Bancinsure Inc',
       'Bell United Ins Co', 'Century-Natl Ins Co', 'Co-Operative Ins Co',
       'Consumers Ins Usa Inc', 'Cornerstone Natl Ins Co',
       'Federated Natl Ins Co', 'First Amer Ins Co',
       'Florists Mut Ins Grp', 'Harbor Ins Co', 'Homestead Ins Co',
       'Inland Mut Ins Co', 'Interstate Auto Ins Co Inc', 'Lancer Ins Co',
       'Lumber Ins Cos', 'Manhattan Re Ins Co', 'Mennonite Mut Ins Co',
       'Middle States Ins Co Inc', 'National Automotive Ins',
       'Nevada General Ins Co', 'New Jersey Citizens United Rcp Exch',
       'Nichido Fire & Marine Ins Co Ltd', 'Northwest Gf Mut Ins Co',
       'Ocean Harbor Cas Ins Co', 'Overseas Partners Us Reins Co',
       'Pacific Ind Ins Co', 'Pacific Pioneer Ins Co',
       'Pacific Specialty Ins Co', 'Penn Miller Grp',
       'Pennsylvania Mfg Asn Ins Co', 'Pioneer State Mut Ins Co',
       'Protective Ins Grp', 'San Antonio Reins Co',
       'Seminole Cas Ins Co', 'Southern Group Ind Inc',
       'Southern Mut Ins Co', 'Southland Lloyds Ins Co', 'Star Ins Grp',
       'Sterling Ins Co', 'Usauto Ins Co', 'Vanliner Ins Co',
       'Wea Prop & Cas Ins Co', 'Wellington Ins Co', 'State Farm Mut Grp', 'United Services Automobile Asn Grp',
       'US Lloyds Ins Co', 'Toa-Re Ins Co Of Amer', 'FL Farm Bureau Grp'])

input = input[~input.GRNAME.isin(cleaning_cond)]

Lista_entidades_ceros = input[input.IncurLoss_B <= 0]["GRCODE"].unique()
input = input[~input.GRCODE.isin(Lista_entidades_ceros)]
#input.IncurLoss_B = input.IncurLoss_B+1 #deal with NaN from log transformation.

In [129]:
df_data        = input #pd.read_csv('medmal_pos.csv')
df_trg_entreno = triangulo(df_data, grcode=43, entreno=True)
df_trg_prueba  = triangulo(df_data, grcode=43, entreno=False)
df_trg_entreno

DevelopmentLag,1,2,3,4,5,6,7,8,9,10
0,607.0,1254.0,1836.0,2434.0,3048.0,3663.0,4278.0,4892.0,5506.0,6120.0
1,2254.0,5113.0,8092.0,10856.0,13682.0,16699.0,19689.0,22667.0,25645.0,NaN
2,5843.0,13267.0,21574.0,30245.0,39311.0,48237.0,57004.0,65769.0,NaN,NaN
3,11422.0,27515.0,46163.0,65258.0,83911.0,102380.0,120787.0,NaN,NaN,NaN
4,19933.0,44095.0,72834.0,101163.0,129234.0,156956.0,NaN,NaN,NaN,NaN
5,24604.0,56734.0,90309.0,123078.0,156800.0,NaN,NaN,NaN,NaN,NaN
6,40735.0,84679.0,127190.0,168902.0,NaN,NaN,NaN,NaN,NaN,NaN
7,43064.0,86769.0,129678.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,41837.0,83141.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,44436.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [130]:
Y = matrix_y(df_trg_entreno)
X = matrix_X(df_trg_entreno)

Y_X          = pd.merge(Y, X, on='y_ii', how='inner')
data_entreno = Y_X[Y_X['Y'].notna()]
data_entreno = data_entreno.drop(['y_ii'], axis=1)
data_entreno.head()

,Y,b0,a_2,a_3,a_4,a_5,a_6,a_7,a_8,a_9,a_10,u_2,u_3,u_4,u_5,u_6,u_7,u_8,u_9,u_10
0,6.408529,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,7.134094,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,7.515345,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,7.797291,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,8.022241,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [132]:
Y_prueba      = matrix_y(df_trg_prueba)                   
x_prueba      = Y_X[Y_X['Y'].notna()].drop(['Y'], axis=1)
data_prueba_  = pd.merge(Y_prueba, x_prueba, on='y_ii', how='inner')
data_prueba=data_prueba_.drop(['y_ii'], axis=1)
y_ii = data_prueba_['y_ii']


In [133]:
x_entreno = data_entreno.drop('Y', axis=1)  # Features
y_entreno = data_entreno['Y']  # Target variable
x_prueba  = data_prueba.drop('Y', axis=1)  # Features
y_prueba  = data_prueba['Y']  # Target variable

In [134]:
Regresion_lineal = LinearRegression()
Regresion_lineal.fit(x_entreno, y_entreno)
LR_coef = Regresion_lineal.coef_
y_pred = Regresion_lineal.predict(x_prueba)

mse = mean_squared_error(y_prueba, y_pred)   # Considerar que se debe aplicar la exponencial a los rsultados
mape = mean_absolute_percentage_error(y_prueba, y_pred)   # COnsiderar que se debe aplicar la exponencial a los rsultados
[mse, mape]

[0.002230701729399499, 0.0038343781268415497]

In [137]:
alpha = 0.00001
ridge_model = Ridge(alpha = alpha) #aplicación regresión de ridge
ridge_model.fit(x_entreno, y_entreno) #entrenamiento regresión de ridge
ridge_coef = ridge_model.coef_ #coeficientes regresión de ridge
y_pred = ridge_model.predict(x_prueba)

mse = mean_squared_error(y_prueba, y_pred)   # COnsiderar que se debe aplicar la exponencial a los rsultados
mape = mean_absolute_percentage_error(y_prueba, y_pred)   # COnsiderar que se debe aplicar la exponencial a los rsultados
[mse, mape]

[0.002230702303162153, 0.003834869524506071]

In [138]:
lasso_model = Lasso(alpha = alpha) #aplicación regresión de lasso
lasso_model.fit(x_entreno, y_entreno) #entrenamiento regresión de lasso
lasso_coef = lasso_model.coef_ #coeficientes regresión de lasso
y_pred = lasso_model.predict(x_prueba)

mse = mean_squared_error(y_prueba, y_pred)   # Considerar que se debe aplicar la exponencial a los rsultados
mape = mean_absolute_percentage_error(y_prueba, y_pred)   # Considerar que se debe aplicar la exponencial a los rsultados
[mse, mape]

[0.0022309333284786215, 0.0038438188771605904]

In [173]:
np.exp(y_pred)

array([   523.69588732,   1138.91125798,   1800.60267558,   2481.25822179,
         3197.47927039,   3891.61694156,   4565.05488313,   5164.35159533,
         5680.89885172,   6130.        ,   2295.86135051,   4992.94037277,
         7893.76848391,  10877.73455923,  14017.61834235,  17060.68950208,
        20013.01389388,  22640.30616852,  24904.82821343,   6338.44464237,
        13784.57638498,  21793.22133009,  30031.39467575,  38700.02770899,
        47101.37916013,  55252.19572384,  62505.6592827 ,  13208.63690653,
        28725.57459323,  45414.72929312,  62582.19649294,  80646.69538382,
        98154.20820492, 115139.6332668 ,  20832.59948126,  45305.8400049 ,
        71627.89564198,  98704.34348534, 127195.58545728, 154808.35164171,
        25717.11423791,  55928.47230123,  88422.12785315, 121847.05415536,
       157018.49424527,  38030.54940143,  82707.20070384, 130758.92070899,
       180187.806828  ,  40167.54770632,  87354.65256788, 138106.47672126,
        39993.8249981 ,  

In [110]:
auto_triangles = cl.Triangle(input, 
                             index = 'GRCODE',
                             origin="AccidentYear",
                             development="DevelopmentYear",
                             columns=["IncurLoss_B"],
                             cumulative=True)

In [67]:
class ChainLadder: #clase se cálcula el Chain-Ladder
    def __init__(self, tabla = pd.DataFrame(), origin = "", development = "", columns = "", index = ""):

        self.tabla = tabla #OK
        self.origin = origin #OK
        self.development = development #OK
        self.index = index #OK
        self.columns = columns #OK

    def Triangulos(self):

        # Renombrar las columnas
        datos = self.tabla.rename(columns={self.origin: "AccidentYear", self.development: "DevelopmentLag", self.columns: "IncurLoss_B",
                                          self.index: "GRCODE"})

        diccionario_todos_triangulos = {}

        for k in datos["GRCODE"].unique():


            Filtro_datos = datos[datos["GRCODE"] == k] #se filtran los datos filtra por aseguradora

            #se crean los triangulos
            Triangulo_full = Filtro_datos.pivot_table(values = "IncurLoss_B", index = "AccidentYear", columns='DevelopmentLag', aggfunc="sum", margins=False)
            #se crea una copia
            Triangulo_full_acumulado = Triangulo_full.copy()
            #se guarda el numero de filas y columnas del triangulo
            num_filas = Triangulo_full_acumulado.shape[0]
            num_columnas = Triangulo_full_acumulado.shape[1]
            #se eliminan los datos del triangulo inferior
            Triangulo_full_mitad = Triangulo_full.copy()
            for i in range(num_filas):
                for j in range(1,i+1):
                    Triangulo_full_mitad.iloc[i, -j] = None  # Puedes establecerlo en None u otro valor si lo prefieres
            #se suman las columnas para hallar el tirangulo acumulado
            for indice, i in enumerate(range(1,num_columnas+1)):
                Triangulo_full_acumulado[Triangulo_full.columns[indice]] = Triangulo_full[Triangulo_full.columns[0:i]].sum(axis = 1)
            #se halla eliminan datos del triangulo inferior del triangulo acumulado
            Triangulo_acumulado_mitad = Triangulo_full_acumulado.copy()
            for i in range(num_filas):
                for j in range(1,i+1):
                    Triangulo_acumulado_mitad.iloc[i, -j] = None  # Puedes establecerlo en None u otro valor si lo prefieres
            #se calculan los factores
            factores0 = Triangulo_acumulado_mitad.sum(axis = 0) # Rojo
            factores1 = Triangulo_acumulado_mitad.sum(axis = 0)-np.flip(np.diag(np.fliplr(Triangulo_acumulado_mitad), 0)) # Azul
            factores0 = factores0[1:10]
            factores1 = factores1[0:-1]
            factores = factores0.reset_index(drop = True) / factores1.reset_index(drop = True)
            #se estima el triangulo por chain-ladder
            Triangulo_estimado = Triangulo_acumulado_mitad.copy()
            for i in list(reversed(range(num_filas))):
                comodin = np.diag(np.fliplr(Triangulo_acumulado_mitad), 0)[i]
                for j in range(1,i+1):
                    Triangulo_estimado.iloc[i, -j] = comodin*factores.iloc[-i+9:-j+10].prod()   # Puedes establecerlo en None u otro valor si lo prefieres


            #se calcula la reserva
            reserva_total = sum(np.array(list(reversed(np.array(Triangulo_estimado[10]))))-np.flip(np.diag(np.fliplr(Triangulo_estimado), 0)))
            #se crea diccionario que va ser el resultado final
            diciconario_triangulo = {'Triangulo_full':Triangulo_full, "Triangulo_full_mitad":Triangulo_full_mitad, "Triangulo_full_acumulado":Triangulo_full_acumulado,
                                     "Triangulo_acumulado_mitad":Triangulo_acumulado_mitad, "factores":factores, "Triangulo_estimado":Triangulo_estimado,
                                     "reserva_total":reserva_total}

            nombre = k
            diccionario_todos_triangulos[nombre] = diciconario_triangulo

        return diccionario_todos_triangulos


In [25]:
class ChainLadder_corto: #clase se cálcula el Chain-Ladder corto
    def __init__(self, tabla = pd.DataFrame(), origin = "", development = "", columns = "", index = ""):

        self.tabla = tabla #OK
        self.origin = origin #OK
        self.development = development #OK
        self.index = index #OK
        self.columns = columns #OK

    def Triangulos(self):

        # Renombrar las columnas
        datos = self.tabla.rename(columns={self.origin: "AccidentYear", self.development: "DevelopmentLag", self.columns: "IncurLoss_B",
                                          self.index: "GRCODE"})

        diccionario_todos_triangulos = {}

        for k in datos["GRCODE"].unique():

            Filtro_datos = datos[datos["GRCODE"] == k]#se filtran los datos filtra por aseguradora
            #se crean los triangulos
            Triangulo_full = Filtro_datos.pivot_table(values = "IncurLoss_B", index = "AccidentYear", columns='DevelopmentLag', aggfunc="sum", margins=False)
            #se crea una copia
            Triangulo_full_acumulado = Triangulo_full.copy()
            #se guarda el numero de filas y columnas del triangulo
            num_filas = Triangulo_full_acumulado.shape[0]
            num_columnas = Triangulo_full_acumulado.shape[1]
            #se eliminan los datos del triangulo inferior
            Triangulo_full_mitad = Triangulo_full.copy()
            for i in range(num_filas):
                for j in range(1,i+1):
                    Triangulo_full_mitad.iloc[i, -j] = None  # Puedes establecerlo en None u otro valor si lo prefieres
            #se crea diccionario que va ser el resultado final
            diciconario_triangulo = {'Triangulo_full':Triangulo_full, "Triangulo_full_mitad":Triangulo_full_mitad}

            nombre = k
            diccionario_todos_triangulos[nombre] = diciconario_triangulo

        return diccionario_todos_triangulos

In [68]:
resultados = ChainLadder(tabla = input, 
                         origin = "AccidentYear", 
                         development = "DevelopmentLag", 
                         columns = input.columns[5], 
                         index = "GRCODE")
triangulos_resultados = resultados.Triangulos()

In [104]:
size = triangulos_resultados[input["GRCODE"].unique()[0]]["Triangulo_acumulado_mitad"].shape[0]#10
size1 =  size**2 #100
size2 = (size1-size)/2 + size #55 
cant_parametros = size*2-1 #19

In [105]:
matriz_de_ceros = np.zeros((size1, cant_parametros)) #matriz de ceros
matriz_de_ceros[:,0] = 1 #columna 0 se llene de unos
for i in range(size): #recorre las columnas
    for j in range(size): #recorre las filas
        k = i*10+j
        if i != 0:            
            matriz_de_ceros[k,i] = 1
        if j != 0:    
            matriz_de_ceros[k,j+9] = 1
matriz_de_ceros = pd.DataFrame(matriz_de_ceros)

Ejemplo de como utilizar la clase para entrenar y evaluar las tres regresiones

In [106]:
df_filtrado = input[input.columns[0:6]]
df_filtrado_unico = df_filtrado[df_filtrado["GRCODE"] == df_filtrado["GRCODE"].unique()[0]] #conjunto de validación (una aseguradora)
df_filtrado_unico1 = df_filtrado[df_filtrado["GRCODE"] == df_filtrado["GRCODE"].unique()[1]] #conjunto de testeo (una aseguradora)
df_filtrado #Dataframe completo con todas las aseguradoras

,GRCODE,GRNAME,AccidentYear,DevelopmentYear,DevelopmentLag,IncurLoss_B
0,43,IDS Property Cas Ins Co,1988,1988,1,607
1,43,IDS Property Cas Ins Co,1988,1989,2,647
2,43,IDS Property Cas Ins Co,1988,1990,3,582
3,43,IDS Property Cas Ins Co,1988,1991,4,598
4,43,IDS Property Cas Ins Co,1988,1992,5,614
...,...,...,...,...,...,...
14571,43494,American Hallmark Ins Co Of TX,1995,1996,2,6128
14572,43494,American Hallmark Ins Co Of TX,1995,1997,3,6320
14580,43494,American Hallmark Ins Co Of TX,1996,1996,1,6554
14581,43494,American Hallmark Ins Co Of TX,1996,1997,2,6907


In [107]:
#Se aplica la clase y se entrenan los modelos
#Acá se entrenan con todas las aseguradoras
resultados1 = Reserva_Regresion_lineal(tabla = df_filtrado, 
                                       origin = "AccidentYear", 
                                       development = "DevelopmentLag", 
                                       columns = df_filtrado.columns[5],
                                       index = "GRCODE",
                                       alpha = 0.001,
                                       matriz_de_ceros = matriz_de_ceros)
resultados_regresion = resultados1.Regresion_lineal()
resultados_regresion["coeficientes"]

{'Coef_normal': array([ 0.        ,  0.00293608,  0.00263851, -0.00548793, -0.00746965,
        -0.01451201, -0.01358531, -0.01566701, -0.01941035, -0.01646579,
         0.22671865,  0.46639164,  0.55386425,  0.56393294,  0.64351185,
         0.79111132,  0.76118006,  0.852593  ,  0.8544819 ]),
 'Coef_ridge1': array([ 0.        ,  0.00293509,  0.00263685, -0.0054901 , -0.00747235,
        -0.01451524, -0.01358917, -0.01567176, -0.01941657, -0.01647572,
         0.22671108,  0.46638327,  0.55385531,  0.56392353,  0.64350175,
         0.79110013,  0.76116791,  0.85257847,  0.85446196]),
 'Coef_lasso': array([ 0.        ,  0.00814668,  0.00473728,  0.        , -0.        ,
        -0.        , -0.        , -0.00092981, -0.00470564, -0.00186443,
         0.17157443,  0.41125028,  0.49872715,  0.50860543,  0.58809926,
         0.73380679,  0.70005797,  0.7821373 ,  0.75913247])}

In [108]:
resultados_regresion["X"]

,u,alpha1,alpha2,alpha3,alpha4,alpha5,alpha6,alpha7,alpha8,alpha9,beta1,beta2,beta3,beta4,beta5,beta6,beta7,beta8,beta9
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4780,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4781,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4782,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4783,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Evaluación del modelo en el conjunto de validación

In [110]:
resultados1.predict(df_filtrado_unico)

Empty DataFrame
Columns: [MAPE]
Index: []


({'MAPE': nan, 'MAPE_ridge_1': nan, 'MAPE_lasso': nan},
 {'MSE': nan, 'MSE_ridge_1': nan, 'MSE_lasso': nan})

In [95]:
reultados_validacion[0]['Metricas MAPE'] #Acá se evaluan los tres modelos en el conjunto de validación o aseguradora de validación
print("sSe evidencia que el mejor modelo por la métrica MAPE es:", reultados_validacion[1]["nombre mejor modelo"])


TypeError: 'NoneType' object is not subscriptable

In [55]:
reultados_validacion = resultados1.predict(df_filtrado_unico)

IndexError: index 0 is out of bounds for axis 0 with size 0

Acá se evaluan los tres modelos en el conjunto de validación o aseguradora de validación


Evaluación del modelo en el conjunto de testeo
Acá se evalua el mejor modelo que escogieron los datos de validación en el conjunto de test o aseguradora de test

In [ ]:
resultados_test = resultados1.predict_test(df_filtrado_unico1)
#Comparación entre los datos reales de la aseguradora de test y los estimados por el modelo final

resultados_test["Comparacion"]

In [185]:
resultados1.predict(input_0)

IndexError: index 0 is out of bounds for axis 0 with size 0